In [1]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict
from scipy.stats.stats import pearsonr

In [2]:
df=pd.read_csv("datatraining.csv", sep=";")

In [3]:
df.head()

,id,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,1,04/02/15 17:51,23.18,27.2720,426.0,721.25,0.004793,1
1,2,04/02/15 17:51,23.15,27.2675,429.5,714.00,0.004783,1
2,3,04/02/15 17:53,23.15,27.2450,426.0,713.50,0.004779,1
3,4,04/02/15 17:54,23.15,27.2000,426.0,708.25,0.004772,1
4,5,04/02/15 17:55,23.10,27.2000,426.0,704.50,0.004757,1


In [4]:
df.isnull().sum()

id               0
date             0
Temperature      0
Humidity         0
Light            0
CO2              0
HumidityRatio    0
Occupancy        0
dtype: int64

In [5]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,id,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
id,1,-0.442176,0.642502,-0.12196,0.122284,0.504377,-0.0981552
Temperature,-0.442176,1,-0.141759,0.649942,0.559894,0.151762,0.53822
Humidity,0.642502,-0.141759,1,0.0378279,0.439023,0.955198,0.132964
Light,-0.12196,0.649942,0.0378279,1,0.664022,0.23042,0.907352
CO2,0.122284,0.559894,0.439023,0.664022,1,0.626556,0.712235
HumidityRatio,0.504377,0.151762,0.955198,0.23042,0.626556,1,0.300282
Occupancy,-0.0981552,0.53822,0.132964,0.907352,0.712235,0.300282,1


In [6]:
df['Occupancy'].value_counts()

0    6414
1    1729
Name: Occupancy, dtype: int64

In [7]:
#Change all the non zero features in 1
df.loc[df['Light'] < 370, 'Light'] = 0
df.loc[df['Light'] >= 370, 'Light'] = 1

In [1]:
df['Light'].value_counts()

NameError: name 'df' is not defined

In [9]:
df['Occupancy'].value_counts()

0    6414
1    1729
Name: Occupancy, dtype: int64

In [10]:
#Crossatab between Occupancy and light
pclass_xt = pd.crosstab(df['Light'], df['Occupancy'])
pclass_xt

Occupancy,0,1
Light,,
0.0,6324,10
1.0,90,1719


In [11]:
#Create a new dataframe with only the division of date and hour
df1=pd.DataFrame({
    'date': pd.to_datetime(df['date']).dt.date,
    'time': pd.to_datetime(df['date']).dt.hour})

In [12]:
#Adding teh previous dataframe to the main dataframe
df['date']=df1['date']
df['Hour']=df1['time']

In [13]:
#The new dataframe with date and time splitted
df.head()

,id,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Hour
0,1,2015-04-02,23.18,27.2720,1.0,721.25,0.004793,1,17
1,2,2015-04-02,23.15,27.2675,1.0,714.00,0.004783,1,17
2,3,2015-04-02,23.15,27.2450,1.0,713.50,0.004779,1,17
3,4,2015-04-02,23.15,27.2000,1.0,708.25,0.004772,1,17
4,5,2015-04-02,23.10,27.2000,1.0,704.50,0.004757,1,17


In [14]:
#Adjusting the order of the column
df=df[['id', 'date', 'Hour', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']]

In [15]:
df.head()

,id,date,Hour,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,1,2015-04-02,17,23.18,27.2720,1.0,721.25,0.004793,1
1,2,2015-04-02,17,23.15,27.2675,1.0,714.00,0.004783,1
2,3,2015-04-02,17,23.15,27.2450,1.0,713.50,0.004779,1
3,4,2015-04-02,17,23.15,27.2000,1.0,708.25,0.004772,1
4,5,2015-04-02,17,23.10,27.2000,1.0,704.50,0.004757,1


In [16]:
#Add the column with the name of the day
df['date'] = pd.to_datetime(df['date'], format='%Y-%d-%m')

df['DayName'] = df['date'].dt.weekday_name

In [17]:
#Adjusting the order of the column
df=df[['id', 'date', 'DayName', 'Hour', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']]

In [18]:
df.head()

,id,date,DayName,Hour,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,1,2015-02-04,Wednesday,17,23.18,27.2720,1.0,721.25,0.004793,1
1,2,2015-02-04,Wednesday,17,23.15,27.2675,1.0,714.00,0.004783,1
2,3,2015-02-04,Wednesday,17,23.15,27.2450,1.0,713.50,0.004779,1
3,4,2015-02-04,Wednesday,17,23.15,27.2000,1.0,708.25,0.004772,1
4,5,2015-02-04,Wednesday,17,23.10,27.2000,1.0,704.50,0.004757,1


In [19]:
#Add a new column for the boolean variable IsWorkDay, this feature is 1 if the day is a work day, 0 otherwise
df["IsWorkDay"] = np.nan
df['IsWorkDay']=df['IsWorkDay'].astype(bool).astype(int)

In [20]:
#Fill the feature IsWorkDay with the corrsponding 1 or 0
for index, row in df.iterrows():
    if row["DayName"]!="Saturday" and row["DayName"]!="Sunday":
        df.loc[index, "IsWorkDay"] = 1
    else:
        df.loc[index, "IsWorkDay"] = 0

In [21]:
#Adjusting the order of the column
df=df[['id', 'date', 'DayName', 'IsWorkDay', 'Hour', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy']]

In [22]:
#Drop the column id because is useless for us
df.drop(['id'], axis=1, inplace =True)

In [23]:
#Drop the column HumidityRatio because is useless for us
df.drop(['HumidityRatio'], axis=1, inplace =True)

In [24]:
#Final attributes of our cleaned dataset
df.head()

,date,DayName,IsWorkDay,Hour,Temperature,Humidity,Light,CO2,Occupancy
0,2015-02-04,Wednesday,1,17,23.18,27.2720,1.0,721.25,1
1,2015-02-04,Wednesday,1,17,23.15,27.2675,1.0,714.00,1
2,2015-02-04,Wednesday,1,17,23.15,27.2450,1.0,713.50,1
3,2015-02-04,Wednesday,1,17,23.15,27.2000,1.0,708.25,1
4,2015-02-04,Wednesday,1,17,23.10,27.2000,1.0,704.50,1


In [25]:
#Add a new column for the variable IsWorkHour, this feature is 1 if the hour is a work hour,
#0 if it isn't a work hour and 2 if we have the doubt, so in the boundary values 
df["IsWorkHour"] = np.nan
df['IsWorkHour']=df['IsWorkHour'].astype(bool).astype(int)

In [26]:
#Fill the feature IsWorkHour with the corrsponding 0, 1 or 2
for index, row in df.iterrows():
    if row["Hour"]==13 or row["Hour"]==7 or row["Hour"]==18:
        df.loc[index, "IsWorkHour"] = 2 #could be both a work hour or not
    elif row["Hour"]<7 or row["Hour"]>18:
        df.loc[index, "IsWorkHour"] = 0 #is not a work hour
    else:
        df.loc[index, "IsWorkHour"] = 1 #is a work hour

In [28]:
#Adjusting the order of the column
df=df[['date', 'DayName', 'IsWorkDay', 'Hour', 'IsWorkHour', 'Temperature', 'Humidity', 'Light', 'CO2', 'Occupancy']]

In [29]:
#test for see if everything works
df[1300:3000]

,date,DayName,IsWorkDay,Hour,IsWorkHour,Temperature,Humidity,Light,CO2,Occupancy
1300,2015-02-05,Thursday,1,15,1,22.500,27.600,1.0,1085.000000,1
1301,2015-02-05,Thursday,1,15,1,22.500,27.700,1.0,1084.000000,1
1302,2015-02-05,Thursday,1,15,1,22.500,27.600,1.0,1076.250000,1
1303,2015-02-05,Thursday,1,15,1,22.500,27.330,1.0,1060.666667,1
1304,2015-02-05,Thursday,1,15,1,22.550,27.445,1.0,1051.500000,1
...,...,...,...,...,...,...,...,...,...,...
2995,2015-02-06,Friday,1,19,0,20.500,19.340,0.0,489.500000,0
2996,2015-02-06,Friday,1,19,0,20.445,19.290,0.0,488.000000,0
2997,2015-02-06,Friday,1,19,0,20.390,19.290,0.0,484.500000,0
2998,2015-02-06,Friday,1,19,0,20.390,19.245,0.0,485.500000,0


In [30]:
#Export the dataset cleaned
df.to_csv(r'DataCleaned.csv')

In [ ]:
df.info()

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')